In [144]:
import oci
import os
from dotenv import load_dotenv, find_dotenv
import numpy as np

In [145]:
_= load_dotenv(find_dotenv())

## OCI認証パラメータの設定

In [146]:
CONFIG_PROFILE = "DEFAULT" # 構成ファイルに合わせて変更してください。
config = oci.config.from_file(file_location='~/.oci/config', profile_name=CONFIG_PROFILE)
config["region"] = "us-chicago-1"

## OCI GenAIクライアントの生成

In [147]:
compartment_id = os.getenv("OCI_COMPARTMENT_ID") 
model_id = os.getenv("OCI_GENAI_EMBED_MODEL_ID")
print(f"model_id:{model_id}")

model_id:cohere.embed-multilingual-v3.0


In [148]:
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

## 埋め込み処理共通のパラメータの設定

In [149]:
embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
embed_text_detail.compartment_id = compartment_id

## 検索対象のドキュメントの定義

In [150]:
'''
documents = [
    "COVID-19 has many symptoms.",
    "COVID-19 symptoms are bad.",
    "COVID-19 symptoms are not nice",
    "COVID-19 symptoms are bad. 5G capabilities include more expansive service coverage, a higher number of available connections, and lower power consumption.",
    "COVID-19 is a disease caused by a virus. The most common symptoms are fever, chills, and sore throat, but there are a range of others.",
    "COVID-19 symptoms can include: a high temperature or shivering (chills); a new, continuous cough; a loss or change to your sense of smell or taste; and many more",
    "Dementia has the following symptom: Experiencing memory loss, poor judgment, and confusion.",
    "COVID-19 has the following symptom: Experiencing memory loss, poor judgment, and confusion.",
]
'''

'\ndocuments = [\n    "COVID-19 has many symptoms.",\n    "COVID-19 symptoms are bad.",\n    "COVID-19 symptoms are not nice",\n    "COVID-19 symptoms are bad. 5G capabilities include more expansive service coverage, a higher number of available connections, and lower power consumption.",\n    "COVID-19 is a disease caused by a virus. The most common symptoms are fever, chills, and sore throat, but there are a range of others.",\n    "COVID-19 symptoms can include: a high temperature or shivering (chills); a new, continuous cough; a loss or change to your sense of smell or taste; and many more",\n    "Dementia has the following symptom: Experiencing memory loss, poor judgment, and confusion.",\n    "COVID-19 has the following symptom: Experiencing memory loss, poor judgment, and confusion.",\n]\n'

In [151]:
documents = [
    "COVID-19の症状は？",
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "近似最近傍探索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。"
]

## 埋め込むドキュメントと生成する埋め込みのタイプ指定

In [152]:
embed_text_detail.inputs = documents
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント

## ドキュメントの埋め込み処理

In [153]:
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)

## クエリーの定義

In [154]:
query = ["COVID-19の症状は？"]
query = ["ベクトルデータベースとは？"]
query = ["ベクトルデータベース、近似最近傍検索とは？"]
query = ["Pineconeとは？"]


## 埋め込むクエリーと生成する埋め込みのタイプ指定

In [155]:
embed_text_detail.inputs = query
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー

## クエリーの埋め込み処理

In [156]:
query_embedding = generative_ai_inference_client.embed_text(embed_text_detail)

## 類似度スコア（コサイン類似度）の計算

In [157]:
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_embedding_vector = np.array(query_embedding.data.embeddings[0])
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

# 類似度スコアの計算（Cohere Embed が生成する埋め込みベクトルはノルムが1になっているため内積を計算するだけでよい）
scores = np.dot(query_embedding_vector, np.transpose(doc_embedding_vectors))

## スコアの表示

In [161]:
import pandas as pd

# スコアとドキュメントをデータフレームに変換
top_n = len(documents)
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})

# データフレームを表示
display(df.style.set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.3813,COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。
1,2,0.3498,新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
2,3,0.3457,COVID-19の症状は？
3,4,0.3433,認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。
4,5,0.3420,COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。
5,6,0.3336,COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
6,7,0.3315,COVID-19の症状は良くありません
7,8,0.3293,近似最近傍探索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
8,9,0.3211,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
9,10,0.3199,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


In [162]:
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .applymap(color_scale, subset=['スコア'])  # スコア列にのみヒートマップを適用
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

C:\Users\yujim\AppData\Local\Temp\ipykernel_69432\3342651954.py:9: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_scale, subset=['スコア'])  # スコア列にのみヒートマップを適用


,ランク,スコア,ドキュメント
0,1,0.3813,COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。
1,2,0.3498,新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
2,3,0.3457,COVID-19の症状は？
3,4,0.3433,認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。
4,5,0.3420,COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。
5,6,0.3336,COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
6,7,0.3315,COVID-19の症状は良くありません
7,8,0.3293,近似最近傍探索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
8,9,0.3211,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
9,10,0.3199,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。
